In [1]:
import pandas as pd
import numpy as np
import random
import datetime

In [2]:
def gerar_dados(tempo_seg):
    dados = {
        'data': [],
        'pedidos_realizados': [],
        'acumulado_pedidos_realizados': [],
        'pedido_em_execucao': [],
        'acumulado_pedidos_finalizados': []
    }
    data = datetime.datetime(2022, 1, 16, 8, 0, 0) # Ano, mês, dia, hora, minuto, segundo
    pedidosRealizados = 0
    pedidosFinalizados = 0  # Começa 
    aux_realizacao = 0
    aux_execucao = 0
    aux_finalizacao = 0
    cont_realizados = 0
    cont_execucao = 0
    cont_finalizados = 0
    execucao = False
    for seg in range(1, tempo_seg + 1):
        
        #Limita a dia de semana
        if data.weekday() >= 5:
            tempoVoltar = 7 - data.weekday()
            data += datetime.timedelta(days=tempoVoltar)

        #Limita ao horario de 8 as 12 13 as 17
        if data.hour >= 12 and data.hour <= 13:
            data += datetime.timedelta(hours=1)
        if data.hour > 17:
            data += datetime.timedelta(hours=15)

        #Limitações logicas sobre entrar em falha e conserto

        aux_realizacao = random.randint(0,9)
        if aux_realizacao == 1:
            pedidosRealizados += 1
            cont_realizados += 1
            #pedidoFinalizado 
            #cont_realizado += 1

        if pedidosRealizados > 0 and execucao == False:
            aux_execucao = random.randint(0,2)
            if aux_execucao == 1 or aux_execucao == 2:
                execucao = True

        if execucao == True:
            if cont_execucao < 12:
                cont_execucao += 1

                # Adiciona os dados para a linha atual no df
                dados['data'].append(data)
                dados['pedidos_realizados'].append(pedidosRealizados)
                dados['acumulado_pedidos_realizados'].append(cont_realizados)
                dados['pedido_em_execucao'].append(execucao)
                dados['acumulado_pedidos_finalizados'].append(cont_finalizados)
                data += datetime.timedelta(seconds=15)

                continue

            else:
                aux_finalizacao = random.randint(0,2)
                if aux_finalizacao == 1 or aux_finalizacao == 2:
                    cont_execucao = 0
                    pedidosRealizados -= 1
                    cont_finalizados += 1
                    execucao = False

        # Adiciona os dados para a linha atual no df
        dados['data'].append(data)
        dados['pedidos_realizados'].append(pedidosRealizados)
        dados['acumulado_pedidos_realizados'].append(cont_realizados)
        dados['pedido_em_execucao'].append(execucao)
        dados['acumulado_pedidos_finalizados'].append(cont_finalizados)
        data += datetime.timedelta(seconds=15)

    return pd.DataFrame(dados)

In [3]:
qtdGerada = 1000000  # Quantidade de dados a gerar por linha, intervalo entre linhas é 15 segundos.

In [10]:
dados_pedidos = gerar_dados(qtdGerada)

In [ ]:
dados_pedidos

In [55]:
dados_pedidos.to_csv('dados_pedidos.csv', index=False)

In [11]:
dados_pedidos["data_abreviada"] = dados_pedidos['data'].dt.strftime('%Y-%m-%d')
dados_pedidos["data_abreviada"] =  pd.to_datetime(dados_pedidos["data_abreviada"])

In [12]:
dados_pedidos['taxa_de_saida(und/h)'] = ''
anos = dados_pedidos['data_abreviada'].dt.year.unique()
meses = dados_pedidos['data_abreviada'].dt.month.unique()
dias = dados_pedidos['data_abreviada'].dt.day.unique()

for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = dados_pedidos[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) & (dados_pedidos['data'].dt.day == dia)]
            try:
                dados_pedidos.loc[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) \
                            & (dados_pedidos['data'].dt.day == dia), "taxa_de_saida(und/h)"] = \
                                (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
                                    )/((df_aux.loc[df_aux['pedido_em_execucao'] == True, 'data'].max() - \
                                    df_aux.loc[df_aux['pedido_em_execucao'] == True,'data'].min()).total_seconds()/3600)
            except ZeroDivisionError:
                 dados_pedidos.loc[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) \
                            & (dados_pedidos['data'].dt.day == dia), "taxa_de_saida(und/h)"] = \
                                (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
                                    )/0.00416666666666666666666666666667



C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\3285628536.py:13: RuntimeWarning: invalid value encountered in divide
  (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\3285628536.py:13: RuntimeWarning: invalid value encountered in divide
  (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\3285628536.py:13: RuntimeWarning: invalid value encountered in divide
  (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\3285628536.py:13: RuntimeWarning: invalid value encountered in divide
  (df_aux.loc[:, 'acumulado_pedidos_finalizados'].max() - df_aux.loc[:, 'acumulado_pedidos_finalizados'].min()\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\3285628536.py:13: Runt

In [13]:
dados_pedidos['takt_time(s/und)'] = ''

anos = dados_pedidos['data_abreviada'].dt.year.unique()
meses = dados_pedidos['data_abreviada'].dt.month.unique()
dias = dados_pedidos['data_abreviada'].dt.day.unique()

for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = dados_pedidos[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) & (dados_pedidos['data'].dt.day == dia)]
            try:
                dados_pedidos.loc[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) \
                            & (dados_pedidos['data'].dt.day == dia), "takt_time(s/und)"] = \
                            ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds())/\
                                (df_aux.loc[:, 'acumulado_pedidos_realizados'].max() - df_aux.loc[:, 'acumulado_pedidos_realizados'].min())
            except ZeroDivisionError:
                 dados_pedidos.loc[(dados_pedidos['data'].dt.year == ano) & (dados_pedidos['data'].dt.month == mes) \
                            & (dados_pedidos['data'].dt.day == dia), "takt_time(s/und)"] = 0

C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\1044519432.py:14: RuntimeWarning: invalid value encountered in divide
  ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds())/\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\1044519432.py:14: RuntimeWarning: invalid value encountered in divide
  ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds())/\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\1044519432.py:14: RuntimeWarning: invalid value encountered in divide
  ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds())/\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\1044519432.py:14: RuntimeWarning: invalid value encountered in divide
  ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds())/\
C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\1044519432.py:14: RuntimeWarning: invalid value encountered in divide
  ((df_aux.loc[:, 'data'].max() - df_aux.loc[:,'data'].min()).total_seconds(

In [14]:
dados_pedidos

,data,pedidos_realizados,acumulado_pedidos_realizados,pedido_em_execucao,acumulado_pedidos_finalizados,data_abreviada,taxa_de_saida(und/h),takt_time(s/und)
0,2022-01-17 08:00:00,0,0,False,0,2022-01-17,13.556485,203.305085
1,2022-01-17 08:00:15,0,0,False,0,2022-01-17,13.556485,203.305085
2,2022-01-17 08:00:30,0,0,False,0,2022-01-17,13.556485,203.305085
3,2022-01-17 08:00:45,0,0,False,0,2022-01-17,13.556485,203.305085
4,2022-01-17 08:01:00,0,0,False,0,2022-01-17,13.556485,203.305085
...,...,...,...,...,...,...,...,...
999995,2024-04-29 09:38:45,28338,99772,True,71434,2024-04-29,16.708861,131.666667
999996,2024-04-29 09:39:00,28339,99773,True,71434,2024-04-29,16.708861,131.666667
999997,2024-04-29 09:39:15,28339,99773,True,71434,2024-04-29,16.708861,131.666667
999998,2024-04-29 09:39:30,28339,99773,True,71434,2024-04-29,16.708861,131.666667


In [15]:
dados_pedidos.loc[:,'taxa_de_saida(und/h)'] = dados_pedidos['taxa_de_saida(und/h)'].fillna(0)
dados_taxa_de_saida = dados_pedidos.groupby(['data_abreviada']).agg({'taxa_de_saida(und/h)': 'mean'})
dados_taxa_de_saida = dados_taxa_de_saida.reset_index()
dados_taxa_de_saida

C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\2638323846.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_pedidos.loc[:,'taxa_de_saida(und/h)'] = dados_pedidos['taxa_de_saida(und/h)'].fillna(0)


,data_abreviada,taxa_de_saida(und/h)
0,2022-01-17,13.556485
1,2022-01-18,13.228346
2,2022-01-19,13.351878
3,2022-01-20,13.339509
4,2022-01-21,13.228346
...,...,...
710,2024-04-24,13.339509
711,2024-04-25,13.339509
712,2024-04-26,13.228346
713,2024-04-27,0.0


In [16]:
dados_pedidos.loc[:,'takt_time(s/und)'] = dados_pedidos['takt_time(s/und)'].fillna(0)
dados_takt_time = dados_pedidos.groupby(['data_abreviada']).agg({'takt_time(s/und)': 'mean'})
dados_takt_time = dados_takt_time.reset_index()
dados_takt_time

C:\Users\jhaid\AppData\Local\Temp\ipykernel_11200\121579202.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_pedidos.loc[:,'takt_time(s/und)'] = dados_pedidos['takt_time(s/und)'].fillna(0)


,data_abreviada,takt_time(s/und)
0,2022-01-17,203.305085
1,2022-01-18,202.40625
2,2022-01-19,197.469512
3,2022-01-20,182.966102
4,2022-01-21,188.284884
...,...,...
710,2024-04-24,199.907407
711,2024-04-25,166.93299
712,2024-04-26,190.5
713,2024-04-27,0.0


In [17]:
dados_taxa_de_saida.to_csv('dados_taxa_de_saida.csv', index=False)
dados_takt_time.to_csv('dados_takt_time.csv', index=False)